# Question 6
We use a Transformer-based architecture with multi-head self-attention and positional encodings, which goes beyond simple MLPs, CNNs, or RNNs.

In [1]:
import os
import tarfile
import glob
import re
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from tqdm import tqdm
import itertools

print("All libraries imported successfully.")


All libraries imported successfully.


Extract and load data.

In [2]:
with tarfile.open("/content/20news-18828.tar.gz") as tar:
    tar.extractall()

def load_20news_data(data_dir='20news-18828'):
    texts = []
    labels = []
    categories = sorted(os.listdir(data_dir))
    for cat in categories:
        cat_dir = os.path.join(data_dir, cat)
        if os.path.isdir(cat_dir):
            for fname in glob.glob(os.path.join(cat_dir, '*')):
                with open(fname, 'r', errors='ignore', encoding='latin1') as f:
                    text = f.read()
                # Remove headers and footers heuristically
                # Remove text before a blank line (headers)
                parts = text.strip().split('\n\n', 1)
                if len(parts) > 1:
                    text_body = parts[1]
                else:
                    text_body = parts[0]
                # Remove quoted lines that start with '>' to reduce noise
                lines = text_body.split('\n')
                lines = [l for l in lines if not l.strip().startswith('>')]
                text_body = ' '.join(lines)
                # Remove non-alphabetic chars
                text_body = re.sub(r'[^a-zA-Z\s]', '', text_body)
                text_body = re.sub(r'\s+', ' ', text_body).strip().lower()

                texts.append(text_body)
                labels.append(cat)
    return texts, labels

print("Loading and preprocessing data...")
texts, labels = load_20news_data()
print("Number of documents:", len(texts))

Loading and preprocessing data...
Number of documents: 18828


Label Encoding and Train-Test Split

In [3]:
label_encoder = LabelEncoder()
y = label_encoder.fit_transform(labels)
X = texts
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
print("Train size:", len(X_train), "Test size:", len(X_test))
print("Number of classes:", len(label_encoder.classes_))

Train size: 15062 Test size: 3766
Number of classes: 20


Tokenization and Vocabulary Building

In [4]:
def tokenize(text):
    return text.strip().split()

# Build a vocabulary of top N words
def build_vocab(texts, vocab_size=20000, min_freq=5):
    freq = {}
    for t in texts:
        for w in tokenize(t):
            freq[w] = freq.get(w, 0) + 1
    # Filter by min_freq
    freq = {w:c for w,c in freq.items() if c >= min_freq}
    # Sort by frequency
    sorted_words = sorted(freq.items(), key=lambda x:x[1], reverse=True)
    sorted_words = [w for w,c in sorted_words[:vocab_size-2]] # reserve spots for PAD and UNK
    vocab = {'<PAD>':0, '<UNK>':1}
    for i,w in enumerate(sorted_words, start=2):
        vocab[w] = i
    return vocab

print("Building vocabulary...")
vocab = build_vocab(X_train)
print("Vocab size:", len(vocab))


Building vocabulary...
Vocab size: 20000


Converting Text to Sequences

In [5]:
def text_to_sequence(text, vocab, max_len=200):
    words = tokenize(text)
    seq = [vocab.get(w, 1) for w in words]  # 1 is <UNK>
    if len(seq) > max_len:
        seq = seq[:max_len]
    else:
        seq += [0]*(max_len-len(seq)) # pad
    return seq

max_len = 200
X_train_seq = [text_to_sequence(t, vocab, max_len) for t in X_train]
X_test_seq = [text_to_sequence(t, vocab, max_len) for t in X_test]

X_train_seq = np.array(X_train_seq, dtype=np.int64)
X_test_seq = np.array(X_test_seq, dtype=np.int64)
y_train = np.array(y_train, dtype=np.int64)
y_test = np.array(y_test, dtype=np.int64)

print("Sequence conversion done. Example:\n", X_train_seq[0][:30])

Sequence conversion done. Example:
 [    9   493  1868  1190    62    14     6     1    15   372 11395    19
   498    45   922   451 10496   977 10496 15838  5395   713  2745     1
     9   446     6   659    12    32]


Dataset and DataLoader

In [6]:
class TextDataset(Dataset):
    def __init__(self, X, y):
        self.X = X
        self.y = y
    def __len__(self):
        return len(self.X)
    def __getitem__(self, idx):
        return torch.tensor(self.X[idx], dtype=torch.long), torch.tensor(self.y[idx], dtype=torch.long)

batch_size = 64
train_dataset = TextDataset(X_train_seq, y_train)
test_dataset = TextDataset(X_test_seq, y_test)

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size)
print("DataLoader ready.")

DataLoader ready.


Define the Transformer-based Model

In [7]:
class PositionalEncoding(nn.Module):
    def __init__(self, d_model, max_len=5000):
        super(PositionalEncoding, self).__init__()
        pe = torch.zeros(max_len, d_model)
        position = torch.arange(0, max_len, dtype=torch.float).unsqueeze(1)
        div_term = torch.exp(torch.arange(0, d_model, 2).float() * -(np.log(10000.0)/d_model))
        pe[:,0::2] = torch.sin(position * div_term)
        pe[:,1::2] = torch.cos(position * div_term)
        pe = pe.unsqueeze(0)
        self.register_buffer('pe', pe)
    def forward(self, x):
        # x: [batch, seq_len, d_model]
        x = x + self.pe[:, :x.size(1), :]
        return x

class MultiHeadSelfAttention(nn.Module):
    def __init__(self, d_model, num_heads):
        super(MultiHeadSelfAttention, self).__init__()
        assert d_model % num_heads == 0
        self.d_k = d_model // num_heads
        self.num_heads = num_heads
        self.w_q = nn.Linear(d_model, d_model)
        self.w_k = nn.Linear(d_model, d_model)
        self.w_v = nn.Linear(d_model, d_model)
        self.fc = nn.Linear(d_model, d_model)

    def forward(self, x, mask=None):
        # x: [batch, seq_len, d_model]
        B, L, D = x.size()
        H = self.num_heads
        q = self.w_q(x).view(B, L, H, self.d_k)
        k = self.w_k(x).view(B, L, H, self.d_k)
        v = self.w_v(x).view(B, L, H, self.d_k)

        # Transpose for attention: [B, H, L, d_k]
        q = q.transpose(1,2)
        k = k.transpose(1,2)
        v = v.transpose(1,2)

        scores = q @ k.transpose(-2, -1) / np.sqrt(self.d_k)
        if mask is not None:
            scores = scores.masked_fill(mask==0, -1e9)
        attn = torch.softmax(scores, dim=-1)
        out = attn @ v
        out = out.transpose(1,2).contiguous().view(B,L,D)
        out = self.fc(out)
        return out

class TransformerEncoderLayer(nn.Module):
    def __init__(self, d_model, num_heads, dim_feedforward=2048, dropout=0.1):
        super(TransformerEncoderLayer, self).__init__()
        self.self_attn = MultiHeadSelfAttention(d_model, num_heads)
        self.linear1 = nn.Linear(d_model, dim_feedforward)
        self.dropout = nn.Dropout(dropout)
        self.linear2 = nn.Linear(dim_feedforward, d_model)

        self.norm1 = nn.LayerNorm(d_model)
        self.norm2 = nn.LayerNorm(d_model)
        self.dropout1 = nn.Dropout(dropout)
        self.dropout2 = nn.Dropout(dropout)

    def forward(self, x, mask=None):
        x2 = self.self_attn(x, mask)
        x = x + self.dropout1(x2)
        x = self.norm1(x)
        x2 = self.linear2(self.dropout(torch.relu(self.linear1(x))))
        x = x + self.dropout2(x2)
        x = self.norm2(x)
        return x

class TransformerClassifier(nn.Module):
    def __init__(self, vocab_size, d_model, num_heads, num_layers, num_classes, max_len, dropout=0.1):
        super(TransformerClassifier, self).__init__()
        self.embedding = nn.Embedding(vocab_size, d_model, padding_idx=0)
        self.pos_enc = PositionalEncoding(d_model, max_len)
        self.layers = nn.ModuleList([
            TransformerEncoderLayer(d_model, num_heads, dim_feedforward=512, dropout=dropout)
            for _ in range(num_layers)
        ])
        self.fc = nn.Linear(d_model, num_classes)
        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        # x: [batch, seq_len]
        emb = self.embedding(x) # [batch, seq_len, d_model]
        emb = self.pos_enc(emb)
        out = emb
        for layer in self.layers:
            out = layer(out)
        # Take the representation of the first token or average pooling
        out = out.mean(dim=1)  # average pooling over time
        out = self.dropout(out)
        logits = self.fc(out)
        return logits

print("Transformer model classes defined.")

Transformer model classes defined.


Instantiate Model, Loss, Optimizer

In [8]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print("Using device:", device)

num_classes = len(label_encoder.classes_)
model = TransformerClassifier(vocab_size=len(vocab), d_model=128, num_heads=4, num_layers=2, num_classes=num_classes, max_len=max_len, dropout=0.1)
model = model.to(device)

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=1e-3)

print("Model, criterion and optimizer are ready.")



Using device: cuda
Model, criterion and optimizer are ready.


Training Loop

In [9]:
epochs = 50
print_every = 100

for epoch in range(epochs):
    model.train()
    total_loss = 0
    for i, (X_batch, y_batch) in enumerate(train_loader):
        X_batch = X_batch.to(device)
        y_batch = y_batch.to(device)

        optimizer.zero_grad()
        logits = model(X_batch)
        loss = criterion(logits, y_batch)
        loss.backward()
        optimizer.step()

        total_loss += loss.item()
        if (i+1) % print_every == 0:
            print(f"Epoch {epoch+1}/{epochs}, Step {i+1}/{len(train_loader)}, Loss: {loss.item():.4f}")
    avg_loss = total_loss / len(train_loader)
    print(f"Epoch {epoch+1} completed. Average Loss: {avg_loss:.4f}")


Epoch 1/50, Step 100/236, Loss: 2.8040
Epoch 1/50, Step 200/236, Loss: 2.4054
Epoch 1 completed. Average Loss: 2.6531
Epoch 2/50, Step 100/236, Loss: 1.6971
Epoch 2/50, Step 200/236, Loss: 1.2132
Epoch 2 completed. Average Loss: 1.6766
Epoch 3/50, Step 100/236, Loss: 1.2525
Epoch 3/50, Step 200/236, Loss: 1.3603
Epoch 3 completed. Average Loss: 1.1367
Epoch 4/50, Step 100/236, Loss: 0.9071
Epoch 4/50, Step 200/236, Loss: 0.5597
Epoch 4 completed. Average Loss: 0.8215
Epoch 5/50, Step 100/236, Loss: 0.8077
Epoch 5/50, Step 200/236, Loss: 0.6414
Epoch 5 completed. Average Loss: 0.6067
Epoch 6/50, Step 100/236, Loss: 0.4702
Epoch 6/50, Step 200/236, Loss: 0.5998
Epoch 6 completed. Average Loss: 0.4324
Epoch 7/50, Step 100/236, Loss: 0.2227
Epoch 7/50, Step 200/236, Loss: 0.2266
Epoch 7 completed. Average Loss: 0.3267
Epoch 8/50, Step 100/236, Loss: 0.1995
Epoch 8/50, Step 200/236, Loss: 0.2399
Epoch 8 completed. Average Loss: 0.2357
Epoch 9/50, Step 100/236, Loss: 0.0848
Epoch 9/50, Step 

Evaluation

In [10]:
model.eval()
correct = 0
total = 0
with torch.no_grad():
    for X_batch, y_batch in test_loader:
        X_batch = X_batch.to(device)
        y_batch = y_batch.to(device)
        logits = model(X_batch)
        preds = torch.argmax(logits, dim=-1)
        correct += (preds == y_batch).sum().item()
        total += y_batch.size(0)

accuracy = correct / total
print(f"Test Accuracy: {accuracy*100:.2f}%")


Test Accuracy: 71.30%
